<a href="https://colab.research.google.com/github/Roudranil/Bayesian-analysis-of-efficacy-of-the-ChAdOx1-nCoV-19-AZD1222-vaccine/blob/main/other_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import numpy as np
import arviz as az

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import graphviz

import scipy
from scipy import stats
from scipy.stats.mstats import mquantiles
from scipy.stats import gaussian_kde as gkde

import theano.tensor as tt

import pymc3 as pm

In [ ]:
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['axes.grid'] = True
az.rcParams['stats.hdi_prob'] = 0.95
%config InlineBackend.figure_format = 'retina'
az.style.use(["arviz-darkgrid", "arviz-orangish"])
mpl.style.use('seaborn-whitegrid')
mpl.rcParams['font.size'] = 14
plt.rcParams["axes.edgecolor"] = "0.15"
plt.rcParams["axes.linewidth"]  = 1.25

# Full bayesian model implementation

In [ ]:
covid_model = pm.Model()

n_v = 17411
n_c = 17411
a_v = 2.428571
a_c = 1
b_v = 0.01917808
b_c = 0.01917808

D = 0.29
n_cases = 170 # 8+162
x_v = 8
x_c = 162

In [ ]:
with covid_model:
    alpha_v = pm.Uniform("alpha_v", lower=0, upper=0.7)
    alpha_c = pm.Uniform("alpha_c", lower=0, upper=0.7)

    lambda_v = pm.Beta("lambda_v", alpha=alpha_v, beta=1)
    lambda_c = pm.Beta("lambda_c", alpha=alpha_c, beta=1)

    exp_sv = n_v*(1 - (1 - tt.exp(-lambda_v*D))/(lambda_v*D))/lambda_v
    var_sv = (2*tt.exp(-lambda_v*D) + (4*tt.exp(-lambda_v*D)/lambda_v*D) - tt.sqr(1 + (tt.exp(-lambda_v*D) - 1)/lambda_v*D))/tt.sqr(lambda_v)
    tau_sv = 1/(n_v*var_sv)

    exp_sc = n_c*(1 - (1 - tt.exp(-lambda_c*D))/(lambda_c*D))/lambda_c
    var_sc = (2*tt.exp(-lambda_c*D) + (4*tt.exp(-lambda_c*D)/lambda_c*D) - tt.sqr(1 + (tt.exp(-lambda_c*D) - 1)/lambda_c*D))/tt.sqr(lambda_c)
    tau_sc = 1/(n_c*var_sc)

    sv = pm.Normal("sv", mu=exp_sv, tau=tau_sv, observed=2214)
    sc = pm.Normal("sc", mu=exp_sc, tau=tau_sc, observed=2222)

    mean_case = sc*lambda_c + sv*lambda_v
    cases = pm.Poisson("cases", mu=mean_case, observed=n_cases)

    theta_v = sv*lambda_v/mean_case
    xv = pm.Binomial("xv", n=cases, p=theta_v, observed=x_v)

    ve = pm.Deterministic("ve", (1 - (lambda_v/lambda_c)))

In [ ]:
with covid_model:
    trace2 = pm.sample(draws=5000, tune=5000, cores=4, return_inferencedata=False, step=pm.NUTS())

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [lambda_c, lambda_v, alpha_c, alpha_v]


Sampling 4 chains for 5_000 tune and 5_000 draw iterations (20_000 + 20_000 draws total) took 51 seconds.


In [ ]:
with covid_model:
    display(az.summary(trace2))

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha_v,0.301,0.171,0.018,0.626,0.001,0.001,19824.0,10338.0,1.0
alpha_c,0.389,0.179,0.097,0.700,0.001,0.001,24065.0,13827.0,1.0
lambda_v,0.004,0.001,0.002,0.007,0.000,0.000,29130.0,14156.0,1.0
lambda_c,0.074,0.006,0.062,0.085,0.000,0.000,24555.0,15004.0,1.0
ve,0.939,0.020,0.900,0.976,0.000,0.000,29319.0,14171.0,1.0


In [ ]:
ve_samp = trace2['ve']
print(f"VE: {100*ve_samp.mean():.2f} +- {100*ve_samp.std():.2f}")
print(f"95% CI: {mquantiles(100*ve_samp, prob=[0.021, 0.979])}")

VE: 93.95 +- 2.00
95% CI: [89.19969788 97.29091607]


In [ ]:
with covid_model:
    display(az.summary(trace1))

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
lambda_v,0.004,0.001,0.002,0.007,0.0,0.0,17344.0,12818.0,1.0
lambda_c,0.074,0.006,0.063,0.085,0.0,0.0,16622.0,14167.0,1.0
ve,0.941,0.020,0.901,0.975,0.0,0.0,17419.0,12598.0,1.0


In [ ]:
with covid_model:
    display(az.summary(trace))

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
lambda_v,0.002,0.001,0.001,0.004,0.0,0.0,17413.0,12020.0,1.0
lambda_c,0.037,0.003,0.031,0.043,0.0,0.0,16046.0,12794.0,1.0
ve,0.937,0.020,0.897,0.975,0.0,0.0,17170.0,12283.0,1.0


In [ ]:
ve_samples_0 = trace['ve']
print(f"VE: {ve_samples_0.mean():.4f} +- {ve_samples_0.std():.4f}")
print(f"95% CI: {mquantiles(ve_samples_0, prob=[0.025, 0.975])}")

VE: 0.9372 +- 0.0205
95% CI: [0.89096621 0.97017505]


In [ ]:
lv = trace['lambda_v']
print(lv.mean())
print(f"{lv.var():.19f}")

0.0023072252453880415
0.0000005197573479206


In [ ]:
f"{0.7/(2214**2):.19f}"

'0.0000001428048005262'

In [ ]:
new_model = pm.Model()

with new_model:
    p_v = pm.Beta("p_v", alpha=0.010101, beta=1)
    p_c = pm.Beta("p_c", alpha=0.010101, beta=1)

    lambda_v = p_v*n_v
    lambda_c = p_c*n_c
    # mean_case = (p_v*n_v + p_c*n_c)
    mean_case = lambda_c + lambda_v
    case = pm.Poisson("cases", mu=mean_case, observed=131)

    theta_v = lambda_v/mean_case
    xv = pm.Binomial("xv", n=cases, p=theta_v, observed=30)

    ve1 = pm.Deterministic("ve1", 1 - (p_v/p_c))
    ve2 = pm.Deterministic("ve2", 1 - (lambda_v/lambda_c))

In [ ]:
with new_model:
    trace_new_0 = pm.sample(draws=5000, tune=5000, cores=4, return_inferencedata=False, step=pm.NUTS())

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p_c, p_v]


Sampling 4 chains for 5_000 tune and 5_000 draw iterations (20_000 + 20_000 draws total) took 35 seconds.


In [ ]:
with new_model:
    display(az.summary(trace_new_0))

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
p_v,0.001,0.000,0.001,0.002,0.0,0.0,15646.0,13256.0,1.0
p_c,0.006,0.001,0.005,0.007,0.0,0.0,15560.0,14307.0,1.0
ve1,0.784,0.044,0.699,0.869,0.0,0.0,15192.0,13746.0,1.0
ve2,0.784,0.044,0.699,0.869,0.0,0.0,15192.0,13746.0,1.0


# full bayesian covishield

covishield data down here

In [ ]:
def model(n_v, n_c, s_v, s_c, D, n_cases, x_v, x_c, prob=0.95, sample_kwargs=None): 
    l = 5/3
    u = 250000
    covid_model = pm.Model()

    with covid_model:
        beta_v = pm.Uniform("beta_v", lower=l, upper=u)
        beta_c = pm.Uniform("beta_c", lower=l, upper=u)

        lambda_v = pm.Gamma("lambda_v", alpha=1, beta=beta_v)
        lambda_c = pm.Gamma("lambda_c", alpha=1, beta=beta_c)

        exp_sv = n_v*(1 - (1 - tt.exp(-lambda_v*D))/(lambda_v*D))/lambda_v
        var_sv = (2*tt.exp(-lambda_v*D) + (4*tt.exp(-lambda_v*D)/lambda_v*D) - tt.sqr(1 + (tt.exp(-lambda_v*D) - 1)/lambda_v*D))/tt.sqr(lambda_v)
        tau_sv = 1/(n_v*var_sv)

        exp_sc = n_c*(1 - (1 - tt.exp(-lambda_c*D))/(lambda_c*D))/lambda_c
        var_sc = (2*tt.exp(-lambda_c*D) + (4*tt.exp(-lambda_c*D)/lambda_c*D) - tt.sqr(1 + (tt.exp(-lambda_c*D) - 1)/lambda_c*D))/tt.sqr(lambda_c)
        tau_sc = 1/(n_c*var_sc)

        sv = pm.Normal("sv", mu=exp_sv, tau=tau_sv, observed=s_v)
        sc = pm.Normal("sc", mu=exp_sc, tau=tau_sc, observed=s_c)

        mean_case = sc*lambda_c + sv*lambda_v
        cases = pm.Poisson("cases", mu=mean_case, observed=n_cases)

        theta_v = sv*lambda_v/mean_case
        xv = pm.Binomial("xv", n=cases, p=theta_v, observed=x_v)
        
        ve = pm.Deterministic("ve", (1 - (lambda_v/lambda_c)))

        trace_covishield = pm.sample(draws=6000, tune=4000, cores=4, return_inferencedata=False, step=pm.NUTS())
        display(az.summary(trace_covishield, hdi_prob=prob))

        ve_samp = trace_covishield['ve']
        print(f"VE: {100*ve_samp.mean():.2f} +- {100*ve_samp.std():.2f}")
        print(f"95% CI: {mquantiles(100*ve_samp, prob=[(1-prob)/2, (1+prob)/2])}")

In [ ]:
params= {"n_v": 5807,
        "n_c": 5829,
        "s_v": 248299,
        "s_c": 247228,
        "D": 0.11829,
        "n_cases": 131,
        "x_v": 30,
        "x_c": 101,
        "prob": 0.958}
        
model(**params)

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [lambda_c, lambda_v, beta_c, beta_v]


Sampling 4 chains for 4_000 tune and 6_000 draw iterations (16_000 + 24_000 draws total) took 47 seconds.


,mean,sd,hdi_2.1%,hdi_97.9%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_v,16851.715,12434.806,350.627,43332.951,83.278,63.734,23050.0,16751.0,1.0
beta_c,4968.434,3566.309,56.133,12395.977,23.163,18.004,22510.0,14210.0,1.0
lambda_v,0.000,0.000,0.000,0.000,0.000,0.000,22262.0,16885.0,1.0
lambda_c,0.000,0.000,0.000,0.000,0.000,0.000,24622.0,17528.0,1.0
ve,0.697,0.063,0.568,0.819,0.000,0.000,22290.0,17060.0,1.0


VE: 69.75 +- 6.34
95% CI: [55.32052429 80.9028787 ]


In [ ]:
cov002_ovr = {"n_v": 3744,
            "n_c": 3804,
            "s_v": 170369,
            "s_c": 170448,
            "D": 0.12542,
            "n_cases": 86,
            "x_v": 18,
            "x_c": 68,
            "prob": 0.95}

model(**cov002_ovr)

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [lambda_c, lambda_v, beta_c, beta_v]


Sampling 4 chains for 4_000 tune and 6_000 draw iterations (16_000 + 24_000 draws total) took 47 seconds.


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_v,19573.617,15050.965,453.980,48933.281,106.214,78.866,20265.0,16285.0,1.0
beta_c,5110.187,3713.641,71.397,12383.597,26.396,19.725,17606.0,12655.0,1.0
lambda_v,0.000,0.000,0.000,0.000,0.000,0.000,21918.0,17830.0,1.0
lambda_c,0.000,0.000,0.000,0.000,0.000,0.000,24055.0,17359.0,1.0
ve,0.726,0.073,0.581,0.858,0.000,0.000,22485.0,17734.0,1.0


VE: 72.56 +- 7.33
95% CI: [56.38228917 84.59602441]


In [ ]:
cov002_ldsd = {"n_v": 1367,
            "n_c": 1374,
            "s_v": 73313,
            "s_c": 72949,
            "D": 0.14822,
            "n_cases": 33,
            "x_v": 3,
            "x_c": 30,
            "prob": 0.95}

model(**cov002_ldsd)

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [lambda_c, lambda_v, beta_c, beta_v]


Sampling 4 chains for 4_000 tune and 6_000 draw iterations (16_000 + 24_000 draws total) took 52 seconds.


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_v,51633.029,44936.991,641.494,148182.519,426.803,321.312,13386.0,11794.0,1.0
beta_c,4960.440,3686.987,82.201,12228.196,28.337,20.038,14516.0,11339.0,1.0
lambda_v,0.000,0.000,0.000,0.000,0.000,0.000,13171.0,15029.0,1.0
lambda_c,0.000,0.000,0.000,0.001,0.000,0.000,15520.0,13189.0,1.0
ve,0.879,0.068,0.745,0.981,0.001,0.000,13046.0,15553.0,1.0


VE: 87.93 +- 6.76
95% CI: [71.40749171 97.00279667]


In [ ]:
cov002_sdsd = {"n_v": 2377,
            "n_c": 2430,
            "s_v": 97056,
            "s_c": 97499,
            "D": 0.11242,
            "n_cases": 53,
            "x_v": 15,
            "x_c": 38,
            "prob": 0.95}

model(**cov002_sdsd)

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [lambda_c, lambda_v, beta_c, beta_v]


Sampling 4 chains for 4_000 tune and 6_000 draw iterations (16_000 + 24_000 draws total) took 47 seconds.


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_v,13368.937,10594.717,265.268,33792.927,80.903,60.546,17039.0,13330.0,1.0
beta_c,5242.803,3917.976,89.189,12935.852,27.811,21.519,19745.0,13823.0,1.0
lambda_v,0.000,0.000,0.000,0.000,0.000,0.000,20396.0,17241.0,1.0
lambda_c,0.000,0.000,0.000,0.001,0.000,0.000,20982.0,16365.0,1.0
ve,0.583,0.129,0.326,0.805,0.001,0.001,20057.0,17145.0,1.0


VE: 58.27 +- 12.89
95% CI: [28.44931225 78.39961711]


In [ ]:
cov003_sdsd = {"n_v": 2063,
            "n_c": 2025,
            "s_v": 77930,
            "s_c": 76780,
            "D": 0.10512,
            "n_cases": 45,
            "x_v": 12,
            "x_c": 33,
            "prob": 0.95}

model(**cov003_sdsd)

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [lambda_c, lambda_v, beta_c, beta_v]


Sampling 4 chains for 4_000 tune and 6_000 draw iterations (16_000 + 24_000 draws total) took 48 seconds.


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_v,13471.439,11008.922,188.554,34821.801,82.758,64.715,18431.0,14581.0,1.0
beta_c,4690.668,3488.650,111.176,11546.996,24.897,18.341,17811.0,13555.0,1.0
lambda_v,0.000,0.000,0.000,0.000,0.000,0.000,21785.0,15751.0,1.0
lambda_c,0.000,0.000,0.000,0.001,0.000,0.000,21764.0,15992.0,1.0
ve,0.619,0.129,0.358,0.839,0.001,0.001,21504.0,16583.0,1.0


VE: 61.89 +- 12.89
95% CI: [31.66055228 81.81524878]


In [ ]:
all_sdsd = {"n_v": 4440,
            "n_c": 4455,
            "s_v": 174986,
            "s_c": 174279,
            "D": 0.10907,
            "n_cases": 98,
            "x_v": 27,
            "x_c": 71,
            "prob": 0.95}

model(**all_sdsd)

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [lambda_c, lambda_v, beta_c, beta_v]


Sampling 4 chains for 4_000 tune and 6_000 draw iterations (16_000 + 24_000 draws total) took 47 seconds.


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_v,13321.240,10035.775,246.685,32808.001,67.429,51.819,22535.0,15589.0,1.0
beta_c,4961.503,3509.439,172.470,11896.623,23.273,17.548,21591.0,14218.0,1.0
lambda_v,0.000,0.000,0.000,0.000,0.000,0.000,22959.0,16607.0,1.0
lambda_c,0.000,0.000,0.000,0.001,0.000,0.000,22907.0,17767.0,1.0
ve,0.611,0.089,0.437,0.773,0.001,0.000,22677.0,17398.0,1.0


VE: 61.14 +- 8.86
95% CI: [41.59556882 75.99681065]


# Modified beta binomial model

In [ ]:
def mod_beta_binomial(n_v, n_c, x_v, x_c, prob):
    l = 0
    u = 0.7
    mod_betabinom = pm.Model()

    with mod_betabinom:
        alpha_v = pm.Uniform("alpha_v", lower=l, upper=u)
        alpha_c = pm.Uniform("alpha_c", lower=l, upper=u)

        v_irr = pm.Beta("v_irr", alpha=alpha_v, beta=1)
        c_irr = pm.Beta("c_irr", alpha=alpha_c, beta=1)

        v_like = pm.Binomial("v_like", n=n_v, p=v_irr, observed=x_v)
        c_like = pm.Binomial("c_like", n=n_c, p=c_irr, observed=x_c)

        ve = pm.Deterministic("ve", (1-(v_irr/c_irr)))

        trace = pm.sample(draws=5000, tune=4000, cores=4, return_inferencedata=False)
        display(az.summary(trace, hdi_prob=prob))

        ve_samp = trace['ve']
        print(f"VE: {100*ve_samp.mean():.2f} +- {100*ve_samp.std():.2f}")
        print(f"95% CI: {mquantiles(100*ve_samp, prob=[(1-prob)/2, (1+prob)/2])}")

In [ ]:
ovr = {"n_v": 5807,
        "n_c": 5829,
        "x_v": 30,
        "x_c": 101,
        "prob": 0.958}

mod_beta_binomial(**ovr)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [c_irr, v_irr, alpha_c, alpha_v]


Sampling 4 chains for 4_000 tune and 5_000 draw iterations (16_000 + 20_000 draws total) took 41 seconds.


,mean,sd,hdi_2.1%,hdi_97.9%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha_v,0.306,0.172,0.022,0.641,0.001,0.001,21331.0,11622.0,1.0
alpha_c,0.346,0.178,0.048,0.677,0.001,0.001,20392.0,11282.0,1.0
v_irr,0.005,0.001,0.003,0.007,0.000,0.000,23459.0,14939.0,1.0
c_irr,0.017,0.002,0.014,0.021,0.000,0.000,25262.0,14600.0,1.0
ve,0.697,0.063,0.566,0.817,0.000,0.000,23724.0,15083.0,1.0


VE: 69.71 +- 6.33
95% CI: [55.3308306  80.82464919]


In [ ]:
cov002_ovr = {"n_v": 3744,
            "n_c": 3804,
            "x_v": 18,
            "x_c": 68,
            "prob": 0.95}

mod_beta_binomial(**cov002_ovr)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [c_irr, v_irr, alpha_c, alpha_v]


Sampling 4 chains for 4_000 tune and 5_000 draw iterations (16_000 + 20_000 draws total) took 40 seconds.


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha_v,0.305,0.173,0.025,0.637,0.001,0.001,21296.0,11875.0,1.0
alpha_c,0.342,0.177,0.044,0.665,0.001,0.001,20471.0,11818.0,1.0
v_irr,0.005,0.001,0.003,0.007,0.000,0.000,26190.0,14934.0,1.0
c_irr,0.018,0.002,0.014,0.022,0.000,0.000,27395.0,14924.0,1.0
ve,0.723,0.074,0.575,0.855,0.000,0.000,26510.0,14564.0,1.0


VE: 72.34 +- 7.42
95% CI: [55.69576573 84.41857249]


In [ ]:
cov002_ldsd = {"n_v": 1367,
            "n_c": 1374,
            "x_v": 3,
            "x_c": 30,
            "prob": 0.95}

mod_beta_binomial(**cov002_ldsd)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [c_irr, v_irr, alpha_c, alpha_v]


Sampling 4 chains for 4_000 tune and 5_000 draw iterations (16_000 + 20_000 draws total) took 40 seconds.


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha_v,0.281,0.167,0.018,0.605,0.001,0.001,18519.0,11554.0,1.0
alpha_c,0.351,0.178,0.077,0.696,0.001,0.001,18760.0,10611.0,1.0
v_irr,0.002,0.001,0.000,0.005,0.000,0.000,21910.0,13416.0,1.0
c_irr,0.022,0.004,0.014,0.030,0.000,0.000,23732.0,15342.0,1.0
ve,0.888,0.066,0.756,0.985,0.000,0.000,21777.0,13215.0,1.0


VE: 88.76 +- 6.64
95% CI: [72.38435516 97.52909054]


In [ ]:
cov002_sdsd = {"n_v": 2377,
               "n_c": 2430,
               "x_v": 15,
               "x_c": 38,
               "prob": 0.95}

mod_beta_binomial(**cov002_sdsd)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [c_irr, v_irr, alpha_c, alpha_v]


Sampling 4 chains for 4_000 tune and 5_000 draw iterations (16_000 + 20_000 draws total) took 41 seconds.


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha_v,0.314,0.173,0.032,0.645,0.001,0.001,22420.0,11510.0,1.0
alpha_c,0.339,0.178,0.049,0.668,0.001,0.001,21980.0,11964.0,1.0
v_irr,0.006,0.002,0.003,0.010,0.000,0.000,25963.0,15209.0,1.0
c_irr,0.016,0.003,0.011,0.021,0.000,0.000,24284.0,14837.0,1.0
ve,0.581,0.129,0.331,0.815,0.001,0.001,25112.0,15806.0,1.0


VE: 58.08 +- 12.90
95% CI: [27.98207512 78.42889347]


In [ ]:
cov003_sdsd = {"n_v": 2063,
               "n_c": 2025,
               "x_v": 12,
               "x_c": 33,
               "prob": 0.95}

mod_beta_binomial(**cov003_sdsd)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [c_irr, v_irr, alpha_c, alpha_v]


Sampling 4 chains for 4_000 tune and 5_000 draw iterations (16_000 + 20_000 draws total) took 40 seconds.


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha_v,0.311,0.174,0.024,0.636,0.001,0.001,18908.0,11526.0,1.0
alpha_c,0.341,0.177,0.055,0.677,0.001,0.001,24080.0,13174.0,1.0
v_irr,0.006,0.002,0.003,0.009,0.000,0.000,21713.0,14761.0,1.0
c_irr,0.016,0.003,0.011,0.022,0.000,0.000,23183.0,15252.0,1.0
ve,0.626,0.128,0.370,0.844,0.001,0.001,22294.0,14211.0,1.0


VE: 62.57 +- 12.79
95% CI: [33.3189964  82.53157533]


In [ ]:
all_sdsd = {"n_v": 4440,
            "n_c": 4455,
            "x_v": 27,
            "x_c": 71,
            "prob": 0.95}

mod_beta_binomial(**all_sdsd)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [c_irr, v_irr, alpha_c, alpha_v]


Sampling 4 chains for 4_000 tune and 5_000 draw iterations (16_000 + 20_000 draws total) took 40 seconds.


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha_v,0.310,0.173,0.020,0.634,0.001,0.001,21399.0,12409.0,1.0
alpha_c,0.341,0.176,0.046,0.667,0.001,0.001,18697.0,11871.0,1.0
v_irr,0.006,0.001,0.004,0.008,0.000,0.000,21404.0,13978.0,1.0
c_irr,0.016,0.002,0.012,0.020,0.000,0.000,26761.0,14413.0,1.0
ve,0.610,0.089,0.432,0.771,0.001,0.000,22544.0,14006.0,1.0


VE: 61.01 +- 8.94
95% CI: [41.57415798 75.97748531]


# Beta-Binomial

In [ ]:
def beta_binomial(n_v, n_c, x_v, x_c, prob):
    l = 0
    u = 0.7
    betabinom = pm.Model()

    with betabinom:
        v_irr = pm.Beta("v_irr", alpha=0.020408, beta=1)
        c_irr = pm.Beta("c_irr", alpha=0.020408, beta=1)

        v_like = pm.Binomial("v_like", n=n_v, p=v_irr, observed=x_v)
        c_like = pm.Binomial("c_like", n=n_c, p=c_irr, observed=x_c)

        ve = pm.Deterministic("ve", (1-(v_irr/c_irr)))

        trace = pm.sample(draws=15000, tune=4000, cores=4, return_inferencedata=False, step=pm.HamiltonianMC())
        display(az.summary(trace, hdi_prob=prob))

        ve_samp = trace['ve'][20000:]
        print(f"VE: {100*ve_samp.mean():.2f} +- {100*ve_samp.std():.2f}")
        print(f"95% CI: {mquantiles(100*ve_samp, prob=[(1-prob)/2, (1+prob)/2])}")

In [ ]:
# advi_map
ovr = {"n_v": 5807,
        "n_c": 5829,
        "x_v": 30,
        "x_c": 101,
        "prob": 0.958}

beta_binomial(**ovr)

Auto-assigning NUTS sampler...
Initializing NUTS using advi_map...


Convergence achieved at 8000
Interrupted at 7,999 [3%]: Average Loss = 25.28
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [c_irr, v_irr]


Sampling 4 chains for 4_000 tune and 5_000 draw iterations (16_000 + 20_000 draws total) took 26 seconds.


,mean,sd,hdi_2.1%,hdi_97.9%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
v_irr,0.005,0.001,0.003,0.007,0.000,0.0,12717.0,10077.0,1.0
c_irr,0.017,0.002,0.014,0.021,0.000,0.0,14003.0,14103.0,1.0
ve,0.698,0.064,0.565,0.821,0.001,0.0,12830.0,11728.0,1.0


VE: 69.80 +- 6.42
95% CI: [55.03365533 80.95972176]


In [ ]:
# advi+adapt_diag

ovr = {"n_v": 5807,
        "n_c": 5829,
        "x_v": 30,
        "x_c": 101,
        "prob": 0.958}

beta_binomial(**ovr)

Auto-assigning NUTS sampler...
Initializing NUTS using advi+adapt_diag...


Convergence achieved at 7900
Interrupted at 7,899 [3%]: Average Loss = 39.917
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [c_irr, v_irr]


Sampling 4 chains for 4_000 tune and 5_000 draw iterations (16_000 + 20_000 draws total) took 27 seconds.


,mean,sd,hdi_2.1%,hdi_97.9%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
v_irr,0.005,0.001,0.003,0.007,0.0,0.0,17797.0,13906.0,1.0
c_irr,0.017,0.002,0.014,0.021,0.0,0.0,16863.0,14039.0,1.0
ve,0.698,0.064,0.569,0.822,0.0,0.0,17980.0,14009.0,1.0


VE: 69.80 +- 6.35
95% CI: [55.13655929 81.02824908]


In [ ]:
# hamiltonian_mc

ovr = {"n_v": 5807,
        "n_c": 5829,
        "x_v": 30,
        "x_c": 101,
        "prob": 0.958}

beta_binomial(**ovr)

Multiprocess sampling (4 chains in 4 jobs)
HamiltonianMC: [c_irr, v_irr]


Sampling 4 chains for 4_000 tune and 15_000 draw iterations (16_000 + 60_000 draws total) took 39 seconds.


,mean,sd,hdi_2.1%,hdi_97.9%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
v_irr,0.005,0.001,0.003,0.007,0.0,0.0,31090.0,34282.0,1.0
c_irr,0.017,0.002,0.014,0.021,0.0,0.0,33737.0,36901.0,1.0
ve,0.698,0.063,0.564,0.816,0.0,0.0,31303.0,33554.0,1.0


VE: 69.83 +- 6.28
95% CI: [55.29922197 80.82648689]


In [ ]:
# slice

ovr = {"n_v": 5807,
        "n_c": 5829,
        "x_v": 30,
        "x_c": 101,
        "prob": 0.958}

beta_binomial(**ovr)

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Slice: [c_irr]
>Slice: [v_irr]


Sampling 4 chains for 4_000 tune and 15_000 draw iterations (16_000 + 60_000 draws total) took 70 seconds.


,mean,sd,hdi_2.1%,hdi_97.9%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
v_irr,0.005,0.001,0.003,0.007,0.0,0.0,59038.0,43609.0,1.0
c_irr,0.017,0.002,0.014,0.021,0.0,0.0,60937.0,44084.0,1.0
ve,0.699,0.063,0.566,0.817,0.0,0.0,58960.0,49075.0,1.0


VE: 69.89 +- 6.28
95% CI: [55.49420444 80.99518992]


# hierarchical beta binomial

So here we have basically the following model structure:

$x_v$ ~ $Bin(n, \theta_v)$

$x_c$ ~ $Bin(n, \theta_c)$

$\theta_v$ ~ $Beta(\alpha, \beta)$

$\theta_c$ ~ $Beta(\alpha, \beta)$

$\mu=\frac{\alpha}{\alpha + \beta}$, [sample mean]

$\eta=\alpha + \beta$, [sample size]


In [ ]:
# overall
n_v = 5807
n_c = 5829
x_v = 30
x_c = 101
prob = 0.958

betabinom = pm.Model()

with betabinom:
    mu = pm.Uniform("mu", lower=0, upper=0.4)
    logeta = pm.Logistic("logeta", mu=np.log(1), s=1)
    eta = pm.Deterministic("eta", tt.exp(logeta))
    # eta = pm.Exponential("eta", lam=0.05)

    alpha = pm.Deterministic("alpha", eta*mu)
    beta = pm.Deterministic("beta", eta*(1-mu))

    theta_v = pm.Beta("theta_v", alpha=alpha, beta=beta)
    theta_c = pm.Beta("theta_c", alpha=alpha, beta=beta)

    X_v = pm.Binomial("X_v", n=n_v, p=theta_v, observed=x_v)
    X_c = pm.Binomial("X_c", n=n_c, p=theta_c, observed=x_c)

    ve = pm.Deterministic("ve", (1-(theta_v/theta_c)))

    hier_trace_ovr = pm.sample(draws=25000, tune=6000, cores=4, return_inferencedata=False, step=pm.HamiltonianMC())
    display(az.summary(hier_trace_ovr, hdi_prob=prob))
    # display(az.plot_trace(hier_trace_ovr))

    ve_samp_ovr = hier_trace_ovr['ve'][20000:]
    print(f"VE: {100*ve_samp_ovr.mean():.2f} +- {100*ve_samp_ovr.std():.2f}")
    print(f"95% CI: {mquantiles(100*ve_samp_ovr, prob=[(1-prob)/2, (1+prob)/2])}")

Multiprocess sampling (4 chains in 4 jobs)
HamiltonianMC: [theta_c, theta_v, logeta, mu]


Sampling 4 chains for 6_000 tune and 25_000 draw iterations (24_000 + 100_000 draws total) took 109 seconds.
There were 227 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.5373842753695052, but should be close to 0.65. Try to increase the number of tuning steps.
There were 122 divergences after tuning. Increase `target_accept` or reparameterize.
There were 39 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.761821814119312, but should be close to 0.65. Try to increase the number of tuning steps.
There were 225 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


,mean,sd,hdi_2.1%,hdi_97.9%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
logeta,0.856,1.339,-1.577,3.918,0.013,0.012,13832.0,8576.0,1.0
mu,0.154,0.104,0.005,0.360,0.001,0.000,16078.0,8299.0,1.0
eta,8.029,28.145,0.029,35.035,0.481,0.340,13832.0,8576.0,1.0
alpha,0.382,0.408,0.009,1.109,0.005,0.004,19742.0,11801.0,1.0
beta,7.648,27.833,0.016,34.123,0.477,0.337,13797.0,8528.0,1.0
theta_v,0.005,0.001,0.003,0.007,0.000,0.000,42372.0,40966.0,1.0
theta_c,0.017,0.002,0.014,0.021,0.000,0.000,40850.0,39417.0,1.0
ve,0.697,0.063,0.566,0.819,0.000,0.000,41994.0,39662.0,1.0


VE: 69.67 +- 6.31
95% CI: [55.26137264 80.82987041]


In [ ]:
# ldsd
n_v = 1367
n_c = 1374
x_v = 3
x_c = 30
prob = 0.95

betabinom = pm.Model()

with betabinom:
    mu = pm.Uniform("mu", lower=0, upper=0.4)
    logeta = pm.Logistic("logeta", mu=np.log(1), s=1)
    eta = pm.Deterministic("eta", tt.exp(logeta))
    # eta = pm.Exponential("eta", lam=0.05)

    alpha = pm.Deterministic("alpha", eta*mu)
    beta = pm.Deterministic("beta", eta*(1-mu))

    theta_v = pm.Beta("theta_v", alpha=alpha, beta=beta)
    theta_c = pm.Beta("theta_c", alpha=alpha, beta=beta)

    X_v = pm.Binomial("X_v", n=n_v, p=theta_v, observed=x_v)
    X_c = pm.Binomial("X_c", n=n_c, p=theta_c, observed=x_c)

    ve = pm.Deterministic("ve", (1-(theta_v/theta_c)))

    hier_trace_ldsd = pm.sample(draws=25000, tune=6000, cores=4, return_inferencedata=False, step=pm.HamiltonianMC())
    display(az.summary(hier_trace_ldsd, hdi_prob=prob))
    # display(az.plot_trace(hier_trace_ldsd))

    ve_samp_ldsd = hier_trace_ldsd['ve'][20000:]
    print(f"VE: {100*ve_samp_ldsd.mean():.2f} +- {100*ve_samp_ldsd.std():.2f}")
    print(f"95% CI: {mquantiles(100*ve_samp_ldsd, prob=[(1-prob)/2, (1+prob)/2])}")

Multiprocess sampling (4 chains in 4 jobs)
HamiltonianMC: [theta_c, theta_v, logeta, mu]


Sampling 4 chains for 6_000 tune and 25_000 draw iterations (24_000 + 100_000 draws total) took 103 seconds.
There were 189 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.4841510440846008, but should be close to 0.65. Try to increase the number of tuning steps.
There were 72 divergences after tuning. Increase `target_accept` or reparameterize.
There were 33 divergences after tuning. Increase `target_accept` or reparameterize.
There were 63 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
logeta,0.641,1.202,-1.582,3.160,0.009,0.007,19096.0,15973.0,1.0
mu,0.159,0.104,0.007,0.358,0.001,0.000,21040.0,17375.0,1.0
eta,4.706,13.046,0.032,17.126,0.113,0.080,19096.0,15973.0,1.0
alpha,0.310,0.274,0.008,0.794,0.002,0.002,30121.0,23575.0,1.0
beta,4.397,12.872,0.023,16.561,0.112,0.079,18817.0,15878.0,1.0
theta_v,0.002,0.001,0.000,0.005,0.000,0.000,36786.0,29514.0,1.0
theta_c,0.022,0.004,0.014,0.030,0.000,0.000,45320.0,40919.0,1.0
ve,0.886,0.067,0.754,0.988,0.000,0.000,36578.0,31584.0,1.0


VE: 88.62 +- 6.72
95% CI: [72.06157646 97.56352198]


In [ ]:
# sdsd
n_v = 4440
n_c = 4455
x_v = 27
x_c = 71
prob = 0.95

betabinom = pm.Model()

with betabinom:
    mu = pm.Uniform("mu", lower=0, upper=0.4)
    logeta = pm.Logistic("logeta", mu=np.log(1), s=1)
    eta = pm.Deterministic("eta", tt.exp(logeta))
    # eta = pm.Exponential("eta", lam=0.05)

    alpha = pm.Deterministic("alpha", eta*mu)
    beta = pm.Deterministic("beta", eta*(1-mu))

    theta_v = pm.Beta("theta_v", alpha=alpha, beta=beta)
    theta_c = pm.Beta("theta_c", alpha=alpha, beta=beta)

    X_v = pm.Binomial("X_v", n=n_v, p=theta_v, observed=x_v)
    X_c = pm.Binomial("X_c", n=n_c, p=theta_c, observed=x_c)

    ve = pm.Deterministic("ve", (1-(theta_v/theta_c)))

    hier_trace_sdsd = pm.sample(draws=25000, tune=6000, cores=4, return_inferencedata=False, step=pm.HamiltonianMC())
    display(az.summary(hier_trace_sdsd, hdi_prob=prob))
    # display(az.plot_trace(hier_trace_sdsd))

    ve_samp_sdsd = hier_trace_sdsd['ve'][20000:]
    print(f"VE: {100*ve_samp_sdsd.mean():.2f} +- {100*ve_samp_sdsd.std():.2f}")
    print(f"95% CI: {mquantiles(100*ve_samp_sdsd, prob=[(1-prob)/2, (1+prob)/2])}")

Multiprocess sampling (4 chains in 4 jobs)
HamiltonianMC: [theta_c, theta_v, logeta, mu]


Sampling 4 chains for 6_000 tune and 25_000 draw iterations (24_000 + 100_000 draws total) took 109 seconds.
There were 85 divergences after tuning. Increase `target_accept` or reparameterize.
There were 418 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.45664911988959356, but should be close to 0.65. Try to increase the number of tuning steps.
There were 85 divergences after tuning. Increase `target_accept` or reparameterize.
There were 50 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7654992713858273, but should be close to 0.65. Try to increase the number of tuning steps.
The number of effective samples is smaller than 10% for some parameters.


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
logeta,0.914,1.405,-1.516,3.999,0.017,0.018,9585.0,4639.0,1.0
mu,0.153,0.105,0.006,0.356,0.001,0.001,13597.0,8642.0,1.0
eta,10.259,38.844,0.018,37.215,0.906,0.640,9585.0,4639.0,1.0
alpha,0.422,0.610,0.004,1.114,0.017,0.015,10398.0,5774.0,1.0
beta,9.837,38.317,0.012,36.446,0.889,0.629,9501.0,4643.0,1.0
theta_v,0.006,0.001,0.004,0.008,0.000,0.000,46387.0,40007.0,1.0
theta_c,0.016,0.002,0.012,0.020,0.000,0.000,44864.0,33153.0,1.0
ve,0.610,0.088,0.432,0.770,0.000,0.000,45400.0,36095.0,1.0


VE: 60.96 +- 8.85
95% CI: [41.21440801 75.80269735]
